In [ ]:
import pandas as pd

In [2]:
data = pd.read_csv("Acute Toxicity_mouse_intraperitoneal_LD50.csv")

In [3]:
data.head()

,TAID,Pubchem CID,IUPAC Name,SMILES,Canonical SMILES,InChIKey,mouse_intraperitoneal_LD50
0,TOX-145,785,"benzene-1,4-diol",Oc1ccc(O)cc1,Oc1ccc(O)cc1,QIGBRXMKCJKVMJ-UHFFFAOYSA-N,3.041835
1,TOX-245,5453,tris(aziridin-1-yl)-sulfanylidene-lambda5-phos...,S=P(N1CC1)(N1CC1)N1CC1,S=P(N1CC1)(N1CC1)N1CC1,FOCVUCIESVLUNU-UHFFFAOYSA-N,4.235584
2,TOX-1273,727,"1,2,3,4,5,6-hexachlorocyclohexane",ClC1C(Cl)C(Cl)C(Cl)C(Cl)C1Cl,ClC1C(Cl)C(Cl)C(Cl)C(Cl)C1Cl,JLYXXMFPNIAWKQ-UHFFFAOYSA-N,3.366732
3,TOX-1279,4091,"3-(diaminomethylidene)-1,1-dimethylguanidine",CN(C)C(=N)N=C(N)N,CN(C)C(=N)N=C(N)N,XZWYZXLIPXDOLR-UHFFFAOYSA-N,2.641604
4,TOX-1282,10364,2-methyl-5-propan-2-ylphenol,Cc1ccc(C(C)C)cc1O,Cc1ccc(C(C)C)cc1O,RECUKUPTGUEGMW-UHFFFAOYSA-N,3.311627


In [6]:
data = data.drop(columns=["TAID","Pubchem CID"])

In [7]:
data.head()

,IUPAC Name,SMILES,Canonical SMILES,InChIKey,mouse_intraperitoneal_LD50
0,"benzene-1,4-diol",Oc1ccc(O)cc1,Oc1ccc(O)cc1,QIGBRXMKCJKVMJ-UHFFFAOYSA-N,3.041835
1,tris(aziridin-1-yl)-sulfanylidene-lambda5-phos...,S=P(N1CC1)(N1CC1)N1CC1,S=P(N1CC1)(N1CC1)N1CC1,FOCVUCIESVLUNU-UHFFFAOYSA-N,4.235584
2,"1,2,3,4,5,6-hexachlorocyclohexane",ClC1C(Cl)C(Cl)C(Cl)C(Cl)C1Cl,ClC1C(Cl)C(Cl)C(Cl)C(Cl)C1Cl,JLYXXMFPNIAWKQ-UHFFFAOYSA-N,3.366732
3,"3-(diaminomethylidene)-1,1-dimethylguanidine",CN(C)C(=N)N=C(N)N,CN(C)C(=N)N=C(N)N,XZWYZXLIPXDOLR-UHFFFAOYSA-N,2.641604
4,2-methyl-5-propan-2-ylphenol,Cc1ccc(C(C)C)cc1O,Cc1ccc(C(C)C)cc1O,RECUKUPTGUEGMW-UHFFFAOYSA-N,3.311627


In [8]:
print(data.isnull().sum())


IUPAC Name                    1919
SMILES                           0
Canonical SMILES                 0
InChIKey                        38
mouse_intraperitoneal_LD50       0
dtype: int64


In [9]:
data = data.dropna

In [13]:
data.head()

AttributeError: 'function' object has no attribute 'head'